In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_LIB = pd.read_csv("/kaggle/input/crystal-system-properties-for-liion-batteries/lithium-ion batteries.csv")

In [ ]:
df_LIB.head()

In [ ]:
plt.bar(df_LIB['Crystal System'], df_LIB['E Above Hull (eV)'])

Orthorhombic crystal has higher E above hull values. #Thermodynamic stable molecules generally has Energy above hull value = 0 meV. Stability decrease with increase in positive values.Let assume that the stable compounds have Ehull values less that 40 meV or 0.04 eV

In [ ]:
df_stable = df_LIB[df_LIB['E Above Hull (eV)']<0.04]

In [ ]:
df_stable

It seems around 93 compounds are thermodynamically stable.

In [ ]:
df_LIB.info()

In [ ]:
df_LIB.describe()

It seems data is normalized as mean is almost equal to median in all variables. However, there is some variation in the values of Nsites and Volume.

In [ ]:
!pip install chemparse
import chemparse

In [ ]:
LIB_dict = df_LIB.Formula.apply(chemparse.parse_formula)
LIB_dict = pd.json_normalize(LIB_dict)
LIB_dict = LIB_dict.fillna(0)
df_LIB = df_LIB.join(LIB_dict)

In [ ]:
df_LIB.head()

In [ ]:
df_LIB = df_LIB.drop(['Materials Id', 'Formula'], axis =1)

In [ ]:
from scipy import stats

In [ ]:
df= pd.crosstab(index =df_LIB['Has Bandstructure'],columns = df_LIB['Crystal System'])

In [ ]:
df

In [ ]:
(chi2, p, dof,_) = stats.chi2_contingency([df.iloc[0].values, df.iloc[1].values])

In [ ]:
print(chi2)

In [ ]:
print(p)

In [ ]:
print(dof)

#since p >0.05 thus, as per null hypothesis there is relationship between the two categorical variables. Now confirm this using Cramers test.

In [ ]:
import numpy as np

In [ ]:
cramer_test = np.sqrt(chi2/df_LIB['Crystal System'].shape[0])

In [ ]:
cramer_test

#Since value is <0.4, means low correlation strength between the two variables.

In [ ]:
df_LIB["Has Bandstructure"] = df_LIB["Has Bandstructure"].map({True:1, False:0})
df_LIB["Crystal System"] = df_LIB["Crystal System"].map({'monoclinic':1, 'orthorhombic':2, 'triclinic':3})

In [ ]:
import seaborn as sns

In [ ]:
corr = df_LIB.corr()
plt.subplots(figsize=(30,30))
sns.heatmap(corr, annot = True)

There is not much correlation between the crystal system and other variables

In [ ]:
!pip install autoviz
!pip install xlrd
#importing Autoviz class
from autoviz.AutoViz_Class import AutoViz_Class

In [ ]:
av = AutoViz_Class()
df_autoviz = av.AutoViz("/kaggle/input/crystal-system-properties-for-liion-batteries/lithium-ion batteries.csv")

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df_LIB, title="Pandas Profiling Report", explorative=True)
profile.to_notebook_iframe()

We have to ignore the outliers in E above hull and density as it seems that they are genuine values corresponding to specific space groups, formation energy.Zero values in E above hull and band gap is also possible. Infact zero values for E above hull means stable crystal.

In [ ]:
!pip install pycaret[full]
from pycaret.regression import *

In [ ]:
df_LIB.Spacegroup = df_LIB.Spacegroup.rank(method="dense").astype(float)
df_LIB['Has Bandstructure'] = df_LIB['Has Bandstructure'].astype(float)
df_LIB['Crystal System'] = df_LIB['Crystal System'].astype(float)
df_LIB['Nsites'] = df_LIB['Nsites'].astype(float)

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
reg_experiment = setup(df_LIB, 
                       target = 'Crystal System', 
                       session_id=123, 
                       log_experiment=True, 
                       experiment_name='pred_CS', remove_multicollinearity = True,
                      multicollinearity_threshold = 0.8, ignore_low_variance = True,
                      feature_selection=True, feature_interaction=True)

In [ ]:
get_config('X')

In [ ]:
top3 = compare_models(n_select = 3)

In [ ]:
gbr = create_model('gbr', cross_validation=True, fold = 10)

In [ ]:
plot_model(gbr, plot='error')

In [ ]:
plot_model(gbr, plot='feature')

# Spacegroup feature is the most import for the prediction of crystal system.

In [ ]:
X = df_LIB.drop(['Crystal System'], axis = 1)
Y = df_LIB['Crystal System']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

In [ ]:
final_gbr = finalize_model(gbr)

In [ ]:
new_predictions = predict_model(gbr, data=X_test)
new_predictions.head()

In [ ]:
df_LIB.loc[df_LIB['Formation Energy (eV)'] == -2.875]

The prediction error is very less and we can see from the data that the crystal system with formation energy of -2.875 eV is predicted correct by the gbr model.